In [7]:
%cd ..

D:\Programming\Research\Thesis\paramsum


In [33]:
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

from collections import defaultdict
from typing import List

import dill

In [53]:
import pandas as pd

import string

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=12)

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

from tqdm.notebook import tqdm

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [54]:
reviews = pd.read_csv('./data/reviews.csv', index_col=0)[['review', 'film_id']]
reviews

,review,film_id
0,«Зеленую милю» я смотрела два раза: 10 лет наз...,435
1,Период конца девяностых годов-начало двухтысяч...,435
2,"Очень сложно писать рецензию на этот фильм, та...",435
3,Любимая многими миллионами ценителями киноиску...,435
4,В нашем мире существует много разных фильмов. ...,435
...,...,...
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...,969760
90643,"Эй, рагацци, вы это серьёзно, ТАК показывать и...",969760
90644,"Вообще, говоря о байопиках, стоит отметить, чт...",969760
90645,"Визуальное качество, впрочем, также не безупре...",969760


In [51]:
reviews = reviews.iloc[:10]
reviews

,review,film_id
0,«Зеленую милю» я смотрела два раза: 10 лет наз...,435
1,Период конца девяностых годов-начало двухтысяч...,435
2,"Очень сложно писать рецензию на этот фильм, та...",435
3,Любимая многими миллионами ценителями киноиску...,435
4,В нашем мире существует много разных фильмов. ...,435
5,В 1999 году этот фильм я и увидел. Его показыв...,435
6,В 1999 году этот фильм я и увидел. Его показыв...,435
7,"Один из лучших фильмов, которые, собственно, я...",435
8,Середина и конец девяностых оказались богатыми...,435
9,"Не читая книгу, мне трудно судить насколько фи...",435


In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [ ]:
def tag_ne(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    
    return doc

In [ ]:
%%time

ne_related_sentences: dict[int, defaultdict[List]] = {}

nents = pd.DataFrame(columns=[0, 1, 'film_id'])

data = reviews
film_ids = data['film_id'].unique()

for i, film_id in enumerate(tqdm(film_ids)):

    dataset = data[data['film_id'] == film_id]
    cur_reviews = dataset['review']
    tagged_reviews = cur_reviews.apply(tag_ne)
    
    for tagged_review in tagged_reviews.values:
        
        if film_id not in ne_related_sentences:
            ne_related_sentences[film_id] = defaultdict(list)
        
        for sent in tagged_review.sents:
            for span in sent.spans:
                if span.type == 'PER':
                    span.normalize(morph_vocab)
                    ne_related_sentences[film_id][span.normal].append(sent.text)
                    # break
               
    new_part = pd.DataFrame(ne_related_sentences[film_id].items())
    new_part['film_id'] = film_id
    nents = pd.concat([nents, new_part], axis=0) 

    
    if i % 100 == 0:
        with open('ne_related_sentences', 'wb') as f:
            dill.dump(nents, f)
    
    # clear_output()
    
    
nents.columns = ['ne', 'occurrences', 'film_id']
nents['n_sents'] = nents['occurrences'].apply(len)
    
    
    
    
    
with open('ne_related_sentences', 'wb') as f:
        dill.dump(nents, f)
        
        

    
nents    

In [64]:
# ~3:50 hours
nents

,ne,occurrences,film_id,n_sents
0,Кинг,"[Я плакала над глубокой и грустной историей, к...",435,187
1,Дарабонт,[После взросления и прочтения книги фильм Дара...,435,83
2,Перси,[Особенно нелепа вся эта линия с травлей Перси...,435,161
3,Депрессия,"[За то, что работает по блату (во времена Депр...",435,1
4,Делакруа,"[И кстати, ради большей слезовыжималки сценари...",435,35
...,...,...,...,...
19,Ферруччо,[Молодой Ферруччо - типичный англо-саксонский ...,969760,10
20,Мазератти,"[Да, Ферруччо Ламборгини был гениальный изобре...",969760,1
21,Эдуардо де Филиппо,[Авторам смотреть старые итальянские фильмы и ...,969760,1
22,Ютюба,[И он по законам мотивашек с Ютюба читает как ...,969760,1


In [69]:
nents['ne'].to_csv('../repos/Attention-Based-Aspect-Extraction/preprocessed_data/review/ne_list.txt', index=False, header=False)
nents['ne']

0                   Кинг
1               Дарабонт
2                  Перси
3              Депрессия
4               Делакруа
             ...        
19              Ферруччо
20             Мазератти
21    Эдуардо де Филиппо
22                 Ютюба
23                  Энцо
Name: ne, Length: 154323, dtype: object

In [16]:
with open('../repos/Attention-Based-Aspect-Extraction/preprocessed_data/review/train.txt', 'wt', encoding='UTF-8') as f:
    for line in tqdm(cur_reviews['review']):
        if len(line.split()) > 1:
            f.write(line + '\n')

  0%|          | 0/2248341 [00:00<?, ?it/s]